## Import modules

In [1]:
import os
import numpy as np
import math
import statistics
import pandas as pd
import random
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

## Load output test probabilitites and true labels for each model

For every model there are 2 files generated in numpy binary format:

  **probabilities.npy** - numpy array of size **TEST POINT CLOUDS CLOUD x CLASSES COUNT x MODELS COUNT** with output classification probability for each of 10 individually trained models

  **true_labels.npy** - numpy array of size **TEST POINT CLOUDS CLOUD** with true labels data

In [2]:
# Base models
BASE_MODELS = ['pointnet', 'pointnet++', 'so-net', 'kcnet', 'deepsets', 'dgcnn', 'pointcnn']
BASE_MODELS_ALIASES = ['PointNet', 'PointNet++', 'SO-Net', 'KCNet', 'DeepSets', 'DGCNN', 'PointCNN']
SONET_BAGGING_MODELS = ['so-net_sz_01', 'so-net_sz_02', 'so-net_sz_03', 'so-net_sz_04', 'so-net_sz_05', 'so-net_sz_06',
                        'so-net_sz_07', 'so-net_sz_08', 'so-net_sz_09', 'so-net_bagging']
SONET_CONST_MODELS = ['so-net_TTT', 'so-net_FFT', 'so-net_FTF', 'so-net_TFF', 'so-net_TTF', 'so-net_TFT', 'so-net_FTT']
SONET_CLASSIFIER_ENSEMBLE = ['so-net_' + str(i) for i in range(1, 11)]
ALL_MODELS = BASE_MODELS + SONET_BAGGING_MODELS + SONET_CONST_MODELS + SONET_CLASSIFIER_ENSEMBLE

CLASS_NAMES = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl',
               'car', 'chair', 'cone', 'cup', 'curtain', 'desk', 'door', 'dresser',
               'flower_pot', 'glass_box', 'guitar', 'keyboard', 'lamp', 'laptop',
               'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 'radio',
               'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet',
               'tv_stand', 'vase', 'wardrobe', 'xbox']
CLASS_TEST_INSTANCES = [100, 50, 100, 20, 100, 100, 20, 100, 100, 20, 20, 20, 86, 20,
                        86, 20, 100, 100, 20, 20, 20, 100, 100, 86, 20, 100, 100, 20,
                        100, 20, 100, 20, 20, 100, 20, 100, 100, 100, 20, 20]
CLASSES_COUNT = 40
# MODELS_COUNT = 10
# ENSEMBLE_MODELS_NUMBER = 10

probabilities = {}
true_labels = {}
for model in ALL_MODELS:
    for dir_name in ['models_reordered', 'models_raw', 'models_bagging', 'models_const', 'models_classifier_ensemble']:
        if os.path.exists(os.path.join(dir_name, model)):
            break
    prob_filepath = os.path.join(os.path.join(dir_name, model), 'probabilities.npy')
    label_filepath = os.path.join(os.path.join(dir_name, model), 'true_labels.npy')
    probabilities[model] = np.load(prob_filepath)
    if dir_name != 'models_reordered':
        probabilities[model] = np.transpose(probabilities[model], axes=(2, 0, 1))

    # probabilities[model] =  softmax(probabilities[model])
    idx = np.argmax(probabilities[model], axis=-1)
    hardvote = np.squeeze(np.eye(40)[idx.reshape(-1)]).reshape(probabilities[model].shape)
    e_x = np.exp(probabilities[model])
    softvote = e_x /np.expand_dims(np.sum(e_x, axis=-1), axis=-1)
    probabilities[model] = {'activations': probabilities[model]/np.std(probabilities[model]),
                           'hardvote': hardvote, 'softvote': softvote}
    true_labels[model] = np.load(label_filepath)
    
    for class_idx in range(CLASSES_COUNT):
        error_info = ("Test cloud instances for model " + model + " and class "
                      + str(class_idx) + " differs! Should be " + str(CLASS_TEST_INSTANCES[class_idx])
                      + " but got " + str(np.sum(true_labels[model] == class_idx)))
        assert np.sum(true_labels[model] == class_idx) == CLASS_TEST_INSTANCES[class_idx] or \
          (model=='deepsets' and class_idx==12 and np.sum(true_labels[model] == class_idx) == 85) or \
          (model=='deepsets' and class_idx==39 and np.sum(true_labels[model] == class_idx) == 19) or \
          (model=='deepsets_ep100' and class_idx==12 and np.sum(true_labels[model] == class_idx) == 85) or \
          (model=='deepsets_ep100' and class_idx==39 and np.sum(true_labels[model] == class_idx) == 19), \
          error_info
                
#     assert (probabilities[model].shape[0] == ENSEMBLE_MODELS_NUMBER)
    assert (probabilities[model]['activations'].shape[1] == 2468 or \
            (model=='deepsets' and probabilities[model]['activations'].shape[1] == 2466) or \
           (model=='deepsets_ep100' and probabilities[model]['activations'].shape[1] == 2466))
    assert (probabilities[model]['activations'].shape[2] == CLASSES_COUNT)

## Caclulate statistics

In [3]:
####################################################################################
# Get random subsets
####################################################################################

def get_k_combinations_of_n_elems(n, k, rep):
    """
    Get rep different k-combinations of set of n numbers. If requested reps is greater
    than possible combinations number just return all possible combinations. In our case:
    n - maximum number of ensembled models
    k - actual number of ensembled models
    rep - how many different k subsets of n we want to generate.    
    """
    c_n_k = math.factorial(n) / math.factorial(k) / math.factorial(n-k)
    combinations_number = min(c_n_k, rep)
    
    subsets = []   
    while len(subsets) != combinations_number:
        candidate = np.sort(np.random.choice(n, k, replace=False))
        if not np.array([(el == candidate).all() for el in subsets]).any():
            subsets.append(candidate)
    return np.array(subsets)
        
####################################################################################
# Calc accuracy statistics
####################################################################################

def get_stats_for_num_ens_param(params_list):
    stats = {}
    stats['mean'] = statistics.mean(params_list)
    stats['stddev'] = statistics.stdev(params_list) if len(params_list) > 1 else 0.0
    stats['median'] = statistics.median(params_list)
    stats['min'] = min(params_list)
    stats['max'] = max(params_list)
    return stats

def get_accuracy_stats(models_list, voting_method='activations'):
    accuracy_stats = {}
    for model in models_list:
        subsets = {}
        
        num_ensemble_models = probabilities[model][voting_method].shape[0]
        for num_ens in range(1, num_ensemble_models+1):
            subset = get_k_combinations_of_n_elems(len(probabilities[model][voting_method]),
                                                   num_ens, 100)
            subsets[num_ens] = subset
            
        accuracy_stats[model] = {}

        for num_ens in subsets:
            num_ens_accs = []
            for subset in subsets[num_ens]:
                preds = np.argmax(np.mean(probabilities[model][voting_method][subset], axis=0), axis=-1)
                true = true_labels[model]

                acc = float(np.sum(preds == true))/len(true)
                class_accs = []
                for class_idx in range(CLASSES_COUNT):
                    class_acc = float(np.sum((preds == true) * (true == class_idx)))/float(np.sum(true == class_idx))
                    class_accs.append(class_acc)
                mean_class_acc = statistics.mean(class_accs)
                num_ens_accs.append({'acc': acc, 'mean_class_acc': mean_class_acc, 'class_accs': class_accs})

            num_ens_summary = {}
            num_ens_summary['acc'] = get_stats_for_num_ens_param([x['acc'] for x in num_ens_accs])
            num_ens_summary['mean_class_acc'] = get_stats_for_num_ens_param([x['mean_class_acc'] for x in num_ens_accs])
            num_ens_summary['class_accs'] = []
            for class_idx in range(CLASSES_COUNT):
                num_ens_summary['class_accs'].append(get_stats_for_num_ens_param([x['class_accs'][class_idx] 
                                                                                  for x in num_ens_accs]))
            accuracy_stats[model][num_ens] = num_ens_summary
    return accuracy_stats

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 16}

plt.rc('font', **font)

def plot_acc_in_num_models(ax, acc_stats, acc_type='acc', styles={}, h_factor=0.1, times=None, without_errorbar=False):
    for model in acc_stats:
        x = acc_stats[model].keys()
        y = [acc_stats[model][i][acc_type]['mean'] for i in x]
        if 'stddev' in acc_stats[model][1][acc_type] and not without_errorbar:
            e = [acc_stats[model][i][acc_type]['stddev'] for i in x]
        if times is not None:
            x = [v * times[model] for v in x]
        if 'stddev' in acc_stats[model][1][acc_type] and not without_errorbar:
            ax.errorbar(x, y, e, label=styles[model]['label'], capsize=6, 
                               color=styles[model]['color'],
                              linestyle=styles[model]['linestyle'])
        else:
            ax.plot(x, y, label=styles[model]['label'], 
                               color=styles[model]['color'],
                              linestyle=styles[model]['linestyle'])
       
    if times is None:
        ax.set_xlabel('Number of ensembled models')
    else:
        ax.set_xlabel('Inference time for batch of 4 point clouds [ms]')
        
    if acc_type == 'acc':
        ax.set_ylabel('Instance classification accuracy')
    elif acc_type == 'mean_class_acc':
        ax.set_ylabel('Mean class accuracy')
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * h_factor,
                     box.width, box.height * (1.0 - h_factor)])

    # Show tick labels
    for tk in ax.get_yticklabels():
        tk.set_visible(True)
        
    return ax.get_legend_handles_labels()

## SO-Net random factors analisys

In [6]:
acc_stats_base_models = get_accuracy_stats(BASE_MODELS, 'activations')

In [4]:
acc_stats_sonet_const = get_accuracy_stats(SONET_CONST_MODELS, 'activations')

In [7]:
data = []
for model in SONET_CONST_MODELS:
    data.append([100 * acc_stats_sonet_const[model][1]['acc']['mean'],
                100 * acc_stats_sonet_const[model][5]['acc']['mean'],
                100 * acc_stats_sonet_const[model][1]['mean_class_acc']['mean'],
                100 * acc_stats_sonet_const[model][5]['mean_class_acc']['mean']])
model = 'so-net'
data.append([100 * acc_stats_base_models[model][1]['acc']['mean'],
                100 * acc_stats_base_models[model][5]['acc']['mean'],
                100 * acc_stats_base_models[model][1]['mean_class_acc']['mean'],
                100 * acc_stats_base_models[model][5]['mean_class_acc']['mean']])
pd.options.display.float_format = '{:,.2f}%'.format
df = pd.DataFrame(data, columns=['Plain acc mean', 'Ensemble acc mean', 
                                 'Plain class acc mean', 'Ensemble class acc mean'], 
                  index=SONET_CONST_MODELS + ['so-net'])
df['Increase acc'] = df['Ensemble acc mean'] - df['Plain acc mean']
df['Increase class acc'] = df['Ensemble class acc mean'] - df['Plain class acc mean']
df

,Plain acc mean,Ensemble acc mean,Plain class acc mean,Ensemble class acc mean,Increase acc,Increase class acc
so-net_TTT,92.34%,93.19%,89.58%,90.96%,0.85%,1.38%
so-net_FFT,92.20%,92.99%,89.62%,90.25%,0.79%,0.63%
so-net_FTF,92.47%,93.35%,89.78%,90.51%,0.88%,0.73%
so-net_TFF,92.49%,93.15%,89.84%,90.65%,0.66%,0.81%
so-net_TTF,92.48%,93.11%,89.89%,90.36%,0.63%,0.48%
so-net_TFT,92.50%,93.15%,90.10%,90.58%,0.65%,0.48%
so-net_FTT,92.51%,93.07%,89.88%,90.40%,0.56%,0.52%
so-net,92.65%,93.57%,89.98%,90.89%,0.92%,0.91%


In [8]:
data = []
TOTAL_NUM_MODELS = 5
for model in SONET_CONST_MODELS:
    data.append([100 * acc_stats_sonet_const[model][1]['acc']['mean'],
                 100 * acc_stats_sonet_const[model][TOTAL_NUM_MODELS]['acc']['mean'],
                 100 * acc_stats_sonet_const[model][1]['mean_class_acc']['mean'],
                 100 * acc_stats_sonet_const[model][TOTAL_NUM_MODELS]['mean_class_acc']['mean']])
model = 'so-net'
data.append([100 * acc_stats_base_models[model][1]['acc']['mean'],
                 100 * acc_stats_base_models[model][TOTAL_NUM_MODELS]['acc']['mean'],
                 100 * acc_stats_base_models[model][1]['mean_class_acc']['mean'],
                 100 * acc_stats_base_models[model][TOTAL_NUM_MODELS]['mean_class_acc']['mean']])
pd.options.display.float_format = '{:,.2f}%'.format
df = pd.DataFrame(data, columns=['Plain acc mean', 'Ensemble acc mean', 
                                 'Plain class acc mean', 'Ensemble class acc mean'], 
                  index=SONET_CONST_MODELS + ['so-net_FFF'])
df['Increase acc'] = df['Ensemble acc mean'] - df['Plain acc mean']
df['Increase class acc'] = df['Ensemble class acc mean'] - df['Plain class acc mean']
df.sort_values(by=['Increase acc'])

,Plain acc mean,Ensemble acc mean,Plain class acc mean,Ensemble class acc mean,Increase acc,Increase class acc
so-net_FTT,92.51%,93.07%,89.88%,90.40%,0.56%,0.52%
so-net_TTF,92.48%,93.11%,89.89%,90.36%,0.63%,0.48%
so-net_TFT,92.50%,93.15%,90.10%,90.58%,0.65%,0.48%
so-net_TFF,92.49%,93.15%,89.84%,90.65%,0.66%,0.81%
so-net_FFT,92.20%,92.99%,89.62%,90.25%,0.79%,0.63%
so-net_TTT,92.34%,93.19%,89.58%,90.96%,0.85%,1.38%
so-net_FTF,92.47%,93.35%,89.78%,90.51%,0.88%,0.73%
so-net_FFF,92.65%,93.57%,89.98%,90.89%,0.92%,0.91%
